<a href="https://githubtocolab.com/geonextgis/geeagri/blob/main/docs/examples/timeseries_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install the latest version of [geeagri](https://geonextgis.github.io/geeagri) if needed.

In [ ]:
# !pip install -U geeagri

## Import libraries

In [ ]:
import ee
import geemap
from geeagri.preprocessing import (
    MeanCentering,
    StandardScaler,
    MinMaxScaler,
    RobustScaler
)
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["font.family"] = "DeJavu Serif"
plt.rcParams["font.serif"] = "Times New Roman"

## Initialize a Map object

In [ ]:
Map = geemap.Map(basemap='SATELLITE')
Map

## Import region of interest

In [ ]:
bbox = [-120.398369, 37.006574, -120.208168, 37.101924]
region = ee.Geometry.BBox(*bbox)
region_style = {"color": "red", "width": 1}
Map.addLayer(region, region_style, "Region")
Map.centerObject(region, 12)

## Load Sentinel-2 Data and Create a Composite

In [ ]:
image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
    .filterBounds(region)\
    .filterDate('2024-01-01', '2025-01-01')\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)\
    .select(['B8', 'B4', 'B3'])\
    .median()\
    .multiply(0.0001)\
    .clip(region)

image_vis = {
    'bands': ['B8', 'B4', 'B3'],
    'min': 0,
    'max': 0.3
}

Map.addLayer(image, image_vis, 'Sentinel 2 FCC')

## Scale the Images

In [ ]:
scaler = RobustScaler(image, region, lower=1, upper=99)
scaled_image = scaler.transform()

scaled_image_vis = {
    'bands': ['B8', 'B4', 'B3'],
    'min': 0,
    'max': 1
}
Map.addLayer(scaled_image, scaled_image_vis, 'RobustScaler')